In [9]:
import chromadb
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
client = chromadb.PersistentClient(path='D:/')

In [12]:
collection = client.get_or_create_collection(
    name="test"
)

In [11]:
import pandas as pd
from tqdm import tqdm


df = pd.read_csv("D:/exdata.csv")

df=df[:1000]


In [13]:
ids = []
metadatas = []
embeddings = []

for row in tqdm(df.iterrows()):
    index = row[0]
    query = row[1].Q
    answer = row[1].A
    
    
    metadata = {
        "query": query,
        "answer": answer
    }
    
    embedding = model.encode(query, normalize_embeddings=True)
    
    ids.append(str(index))
    metadatas.append(metadata)
    embeddings.append(embedding)
embeddings = [ e.tolist() for e in embeddings]
collection.add(embeddings=embeddings, ids=ids, metadatas=metadatas)         

1000it [00:32, 30.87it/s]


In [14]:
result = collection.query(
    query_embeddings=model.encode("금수저 부럽다", normalize_embeddings=True).tolist(),
    n_results=3
)
print(result)

{'ids': [['344', '343', '345']], 'distances': [[0.6046340465545654, 0.6892774105072021, 0.742411732673645]], 'metadatas': [[{'answer': '아이를 금수저로 만들어주세요.', 'query': '금수저로 태어났으면'}, {'answer': '뭔가 안풀리는 일이 있나봐요.', 'query': '금수저 물고 태어나면 좋겠지?'}, {'answer': '아이를 금수저로 만들어주세요.', 'query': '금수저로 태어났으면 좋았을텐데'}]], 'embeddings': None, 'documents': [[None, None, None]], 'uris': None, 'data': None}


In [15]:
for i in result['metadatas'][0]:
    print(i['answer'])

아이를 금수저로 만들어주세요.
뭔가 안풀리는 일이 있나봐요.
아이를 금수저로 만들어주세요.
